In [1]:
import pandas as pd
import itertools
import sys 
import os, glob
import re

In [2]:
sys.path.append('/Users/xehu/Desktop/Team Process Map/team-process-map/feature_engine/')

In [3]:
from utils.preprocess import *
from features.gini_coefficient import *

In [4]:
# test file
INPUT_FILE_PATH = '../../data/raw_data/csop_conversations_withblanks.csv'
chat_data = pd.read_csv(INPUT_FILE_PATH)

chat_data['message'] = chat_data['message'].astype(str).apply(preprocess_text)

In [5]:
chat_data.head()

,batch_num,round_num,round_index,task_index,complexity,type,social_perceptiveness,skill,speaker_nickname,message,normalized_score,zscore_score,zscore_round_duration,zscore_efficiency
0,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,eQuK9PirA9WSa8DrP,well theres nothing i can do about blue moving...,99.10847,0.911454,0.382625,-0.437154
1,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,cRJk4WEXcXPYQxkCJ,my point,99.10847,0.911454,0.382625,-0.437154
2,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,c24Dw7YjjjdNbABLE,now that all of the pieces have been placed in...,99.10847,0.911454,0.382625,-0.437154
3,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,c24Dw7YjjjdNbABLE,conflicts eliminated is there a better combina...,99.10847,0.911454,0.382625,-0.437154
4,C48peJSzquNCHsKSv,8m7bMTXhQH8pEEMfP,2,1,Very low,HH,30.333333,1051.666667,cRJk4WEXcXPYQxkCJ,maybe b and e to102 and 103,100.00000,0.662657,1.111708,-1.120019


In [6]:
chat_data_with_conversation_num = preprocess_conversation_columns(chat_data)

Confirmed that data has `conversation_num` and `message` columns!


In [7]:
chat_data_with_conversation_num

,conversation_num,batch_num,round_num,round_index,task_index,complexity,type,social_perceptiveness,skill,speaker_nickname,message,normalized_score,zscore_score,zscore_round_duration,zscore_efficiency
0,182,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,eQuK9PirA9WSa8DrP,well theres nothing i can do about blue moving...,99.108470,0.911454,0.382625,-0.437154
1,182,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,cRJk4WEXcXPYQxkCJ,my point,99.108470,0.911454,0.382625,-0.437154
2,182,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,c24Dw7YjjjdNbABLE,now that all of the pieces have been placed in...,99.108470,0.911454,0.382625,-0.437154
3,182,C48peJSzquNCHsKSv,e7uDqY9mpTEZH5Q2b,1,4,High,HH,30.333333,1051.666667,c24Dw7YjjjdNbABLE,conflicts eliminated is there a better combina...,99.108470,0.911454,0.382625,-0.437154
4,180,C48peJSzquNCHsKSv,8m7bMTXhQH8pEEMfP,2,1,Very low,HH,30.333333,1051.666667,cRJk4WEXcXPYQxkCJ,maybe b and e to102 and 103,100.000000,0.662657,1.111708,-1.120019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4018,234,DhC75b28nAc9y5W9x,iMW6BXkH9jvcXnLgY,1,5,Very high,LL,25.333333,973.333333,NULL_SPEAKER,nan,78.815261,-0.428645,-1.261356,1.342403
4019,233,DhC75b28nAc9y5W9x,HPxvosEMmRzqt526L,2,3,Moderate,LL,25.333333,973.333333,NULL_SPEAKER,nan,90.922619,0.209227,-0.783813,0.648438
4020,232,DhC75b28nAc9y5W9x,FEqJ3gzQYMK4hPsoz,3,2,Low,LL,25.333333,973.333333,NULL_SPEAKER,nan,100.000000,0.832667,0.746904,-0.815849
4021,230,DhC75b28nAc9y5W9x,9ZhXim8L3v2NbZQsy,4,1,Very low,LL,25.333333,973.333333,NULL_SPEAKER,nan,97.941176,0.452398,-0.250932,-0.188037
